In [16]:
# DOwnload the onnx model from hugging face (configs too)
#  Then downgrade a version because the used triton version does not support the ONNX IR version 10

import onnx

# Load the ONNX model
model_path = "/workspace/onnx-nomic-embed-v1.5/model.onnx"
onnx_model = onnx.load(model_path)

# Set IR version to 9
onnx_model.ir_version = 9

# Save the updated model
onnx.save(onnx_model, "/workspace/onnx-nomic-embed-v1.5/model_ir9.onnx")

print("Successfully downgraded ONNX IR version to 9!")


Successfully downgraded ONNX IR version to 9!


In [17]:
import onnxruntime as ort
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoImageProcessor
from PIL import Image
import requests

# Load the image processor using the preprocessor config
processor = AutoImageProcessor.from_pretrained("/workspace/onnx-nomic-embed-v1.5")

# Load the ONNX model
onnx_model_path = "/workspace/onnx-nomic-embed-v1.5/model.onnx"
session = ort.InferenceSession(onnx_model_path)

# Load an image
url = 'https://m.media-amazon.com/images/I/61sADwl+YWL._AC_UL320_.jpg'
image = Image.open(requests.get(url, stream=True).raw)

# Preprocess the image
inputs = processor(image, return_tensors="np")
onnx_inputs = {session.get_inputs()[0].name: inputs["pixel_values"]}

# Run inference
onnx_outputs = session.run(None, onnx_inputs)

# Extract and normalize embeddings
img_emb = torch.tensor(onnx_outputs[0])
img_embeddings = F.normalize(img_emb[:, 0], p=2, dim=1)

# print("Image Embeddings:", img_embeddings.shape)
img_embeddings

tensor([[-5.9561e-02,  1.4260e-02, -4.2722e-02,  1.7564e-02, -5.2930e-03,
         -1.6759e-02, -1.1354e-02, -2.8022e-02,  2.3635e-03, -5.4893e-02,
         -5.2249e-02, -1.5242e-02, -3.9496e-02, -5.9147e-02, -4.1323e-02,
         -5.5929e-03, -3.6168e-02,  2.5309e-02, -1.5952e-02, -7.5792e-02,
         -3.2261e-02, -4.3237e-02, -8.0759e-03,  1.4871e-03, -5.7533e-02,
         -3.4277e-02, -2.9301e-03, -1.3200e-02,  2.7503e-02, -1.5425e-02,
         -8.4125e-03, -3.0009e-02, -4.8950e-02, -1.0887e-02, -3.4873e-02,
         -3.2400e-02, -2.8373e-02, -5.8485e-02,  2.0070e-02, -3.7936e-02,
         -5.7473e-03, -1.3793e-02, -2.3495e-02, -2.1661e-02,  1.9539e-02,
         -2.6626e-02,  9.3902e-03, -2.5531e-02, -1.4387e-02, -1.2081e-02,
         -1.3193e-02, -6.8248e-02,  1.4177e-02, -2.6486e-02, -7.3345e-02,
         -2.8795e-02, -2.4718e-02, -5.4192e-02, -3.5064e-02, -3.1386e-02,
          4.9922e-03, -6.3652e-03, -4.3442e-02, -1.0965e-02, -1.6557e-02,
         -3.5801e-02,  1.5818e-02,  5.